### 交叉验证 手动分割数据

In [15]:

import torch
import torch.nn as nn
import numpy as np


In [16]:

import  seaborn as sns
iris =sns.load_dataset('iris')
data = torch.tensor(iris[iris.columns[0:4]].values).float()
labels = torch.zeros(len(data),dtype=torch.long)

labels[iris.species == 'versicolor'] = 1
labels[iris.species == 'virginica'] = 2


In [17]:
# 训练数据集比例
propTraining = .8
nTraining = int(len(labels) * propTraining)

# 初始化一个布尔值 用于选择 数据和标签
trainingBool = np.zeros(len(labels),dtype=bool)

# 这样选 合适吗？
# trainingBool[range(nTraining)] = True

## 这个方法会更好一点
items2use4train = np.random.choice(range(len(labels)),nTraining,replace=False)

trainingBool[items2use4train] = True

print(trainingBool)


[ True  True  True  True False False False False  True False False  True
 False False  True  True  True  True  True  True  True  True  True  True
  True False  True  True  True  True False  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True False  True  True  True  True  True  True  True  True  True  True
 False  True  True  True False  True  True  True False  True  True False
  True  True  True  True  True  True  True  True False  True  True  True
  True  True  True False  True  True  True  True False  True False False
  True  True  True  True  True False  True  True  True  True False  True
 False  True  True  True  True  True  True  True  True False  True  True
 False False  True  True  True  True  True  True False  True  True  True
  True  True  True  True  True False False  True  True  True  True  True
 False  True  True  True  True  True]


In [18]:


# = 1  根据定义
print(torch.mean(labels.float()))

# 应该等于1
print(torch.mean(labels[trainingBool].float()))

# 也应该等于1
print(torch.mean(labels[~trainingBool].float()))


tensor(1.)
tensor(1.0083)
tensor(0.9667)


In [19]:

ANNiris = nn.Sequential(
    nn.Linear(4,64),
    nn.ReLU(),
    nn.Linear(64,64),
    nn.ReLU(),
    nn.Linear(64,3)
)

lossfun = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(ANNiris.parameters(),lr=.01)


print(data.shape)

print(data[trainingBool,:].shape)
print(data[~trainingBool,:].shape)

torch.Size([150, 4])
torch.Size([120, 4])
torch.Size([30, 4])


In [21]:

## train and test the model

numepoches = 1000

losses = torch.zeros(numepoches)
ongoingAcc = []

for epochi in range(numepoches):
    
    yHat = ANNiris(data[trainingBool,:])
    ongoingAcc.append(100*torch.mean((
        torch.argmax(yHat,axis=1) ==labels[trainingBool]).float()))

    loss = lossfun(yHat,labels[trainingBool])
    losses[epochi] =loss
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()



In [27]:


predicitions = ANNiris(data[trainingBool,:])

trainacc = 100 * torch.mean(
    (torch.argmax(predicitions,axis=1) == labels[trainingBool].float()))


# predicitions = ANNiris(data[~trainingBool,:])

# testacc = 100 * torch.mean((torch.argmax(predicitions,axix =1) ==labels[~trainingBool]).float())


RuntimeError: mean(): could not infer output dtype. Input dtype must be either a floating point or complex dtype. Got: Bool